# Explore here

In [4]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd

In [5]:
# Parámetros para una primera consulta
fda_key = os.getenv("OPENFDA_API_KEY")

url = "https://api.fda.gov/drug/event.json"

params = {
    "api_key": fda_key,
    "search": "acetaminophen",  # ejemplo: buscar eventos relacionados con acetaminophen
    "limit": 5  
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print("Respuesta API:", data)
else:
    print("Error:", response.status_code, response.text)

Respuesta API: {'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.', 'terms': 'https://open.fda.gov/terms/', 'license': 'https://open.fda.gov/license/', 'last_updated': '2025-04-28', 'results': {'skip': 0, 'limit': 5, 'total': 973540}}, 'results': [{'safetyreportversion': '2', 'safetyreportid': '10003319', 'primarysourcecountry': 'US', 'occurcountry': 'US', 'transmissiondateformat': '102', 'transmissiondate': '20141212', 'reporttype': '1', 'serious': '1', 'seriousnesshospitalization': '1', 'receivedateformat': '102', 'receivedate': '20140312', 'receiptdateformat': '102', 'receiptdate': '20140408', 'fulfillexpeditecriteria': '1', 'companynumb': 'US-GLAXOSMITHKLINE-A1063552A', 'duplicate': '1', 'reportduplicate': {'duplicatesource': 'GLAXOSMITHKLINE', 'd

In [ ]:
#Extracción de datos as DataFrame

datos = []
for report in data['results']:
    paciente = report.get('patient', {})
    reacciones = paciente.get('reaction', [])
    drogas = paciente.get('drug', [])
    
    # Algunas variables pueden ser listas, maneja la primera o concatenalas
    primera_reaccion = reacciones[0]['reactionmeddrapt'] if reacciones else None
    primer_medicamento = drogas[0]['medicinalproduct'] if drogas else None
    
    fila = {
    'safetyreportid': report.get('safetyreportid'),
    'primarysourcecountry': report.get('primarysourcecountry'),
    'occurcountry': report.get('occurcountry'),
    'transmissiondate': report.get('transmissiondate'),
    'receivedate': report.get('receivedate'),
    'serious': report.get('serious'),
    'seriousnesshospitalization': report.get('seriousnesshospitalization'),
    'patientonsetage': paciente.get('patientonsetage'),
    'patientonsetageunit': paciente.get('patientonsetageunit'),
    'patientsex': paciente.get('patientsex'),
    'reactionmeddrapt': primera_reaccion,
    'reactionoutcome': reacciones[0].get('reactionoutcome') if reacciones else None,
    'drugcharacterization': drogas[0].get('drugcharacterization') if drogas else None,
    'medicinalproduct': primer_medicamento,
    'drugauthorizationnumb': drogas[0].get('drugauthorizationnumb') if drogas else None,
    'drugindication': drogas[0].get('drugindication') if drogas else None,
    'drugdosageform': drogas[0].get('drugdosageform') if drogas else None,
    'drugadministrationroute': drogas[0].get('drugadministrationroute') if drogas else None,
    'fulfillexpeditecriteria': report.get('fulfillexpeditecriteria'),
    'companynumb': report.get('companynumb')
}
    datos.append(fila)

df = pd.DataFrame(datos)

  safetyreportid primarysourcecountry occurcountry transmissiondate  \
0       10003319                   US           US         20141212   
1       10003321                   JP           JP         20151125   
2       10003329                   IT           IT         20150821   
3       10003349                   US           US         20141002   
4       10003470                   US           US         20151125   

  receivedate serious seriousnesshospitalization patientonsetage  \
0    20140312       1                          1              46   
1    20140312       1                          1              71   
2    20140312       1                          1              79   
3    20140312       1                          1            None   
4    20140312       2                       None              63   

  patientonsetageunit patientsex                  reactionmeddrapt  \
0                 801          2                   Cholecystectomy   
1                 801   

In [7]:
df.head()

,safetyreportid,primarysourcecountry,occurcountry,transmissiondate,receivedate,serious,seriousnesshospitalization,patientonsetage,patientonsetageunit,patientsex,reactionmeddrapt,reactionoutcome,drugcharacterization,medicinalproduct,drugauthorizationnumb,drugindication,drugdosageform,drugadministrationroute,fulfillexpeditecriteria,companynumb
0,10003319,US,US,20141212,20140312,1,1,46,801,2,Cholecystectomy,1,1,BENLYSTA,125370,SYSTEMIC LUPUS ERYTHEMATOSUS,POWDER FOR INFUSION,042,1,US-GLAXOSMITHKLINE-A1063552A
1,10003321,JP,JP,20151125,20140312,1,1,71,801,1,Adenoviral haemorrhagic cystitis,3,2,ACETAMINOPHEN.,None,None,None,065,1,JP-GLAXOSMITHKLINE-B0975691A
2,10003329,IT,IT,20150821,20140312,1,1,79,801,2,Diarrhoea,1,2,PARACETAMOLO,None,PAIN,TABLET,065,1,"IT-B.I. PHARMACEUTICALS,INC./RIDGEFIELD-2014-B..."
3,10003349,US,US,20141002,20140312,1,1,None,None,2,Cerebrovascular accident,6,1,MORPHINE SULFATE EXTENDED-RELEASE TABLETS (RHO...,074862,None,PROLONGED-RELEASE TABLET,048,1,US-PURDUE-USA-2014-0111828
4,10003470,US,US,20151125,20140312,2,None,63,801,2,Drug ineffective,6,2,FUROSEMIDE.,None,None,None,None,2,US-GILEAD-2012-0059146
